## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [3]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [8]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [48]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3),activation='relu',input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(input_shape=(32,32,3)))

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(Convolution2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))

#卷積組合
#classifier.add(Convolution2D(32, kernel_size=(3, 3),activation='relu',input_shape=(32,3,3)))
#classifier.add(BatchNormalization())
#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(32,input_shape=(32,32,3),activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(units=10,activation='softmax',output_dim=10))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

#score = classifier.evaluate(x_train, y_train, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

TypeError: For the `units` argument, the layer received both the legacy keyword argument `output_dim` and the Keras 2 keyword argument `units`. Stick to the latter!

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [41]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)


array([[0.12845597, 0.08790552, 0.0899328 , 0.06126681, 0.05012718,
        0.06905767, 0.14704037, 0.17430548, 0.10585271, 0.08605546]],
      dtype=float32)